# Geomagnetic Ground Observatory Data

> Authors: Ashley Smith
>
> Abstract: Demonstrates ground observatory data accessed through VirES - this is the AUX_OBS product distributed by BGS to support the Swarm mission, and contains data from INTERMAGNET and the WDC.

WARNING: This is the early release (staging).

In [ ]:
!pip install git+https://github.com/ESA-VirES/VirES-Python-Client.git@staging

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
from viresclient import SwarmRequest

swarm = SwarmRequest('https://staging.viresdisc.vires.services/ows')

### Data are organised into AUX_OBSM (minute) and AUX_OBSS (second) types

In [ ]:
print(swarm.available_collections("AUX_OBSM", details=False))
print(swarm.available_collections("AUX_OBSS", details=False))

In [ ]:
print(swarm.available_measurements("SW_OPER_AUX_OBSM2_"))
print(swarm.available_measurements("SW_OPER_AUX_OBSS2_"))

### Data can be requested similarly to Swarm MAG products

**Note that the IAGA_code variable is necessary in order to distinguish records**

**Note that there is a special message issued regarding the data terms**

In [ ]:
swarm.set_collection("SW_OPER_AUX_OBSM2_")
swarm.set_products(["IAGA_code", "B_NEC", "F", "Quality"])
data = swarm.get_between("2016-01-01", "2016-01-03")
ds = data.as_xarray()
ds

### Use `SwarmRequest.available_observatories()` to find possible IAGA codes

In [ ]:
swarm.available_observatories("SW_OPER_AUX_OBSM2_", details=True)

### IAGA codes can be used to directly fetch specific observatories

Subset the collection with a special collection name like `"SW_OPER_AUX_OBSM2_:<IAGA_code>"` to get data from only that observatory.

In [ ]:
swarm.set_collection("SW_OPER_AUX_OBSM2_:ABK")
swarm.set_products(["B_NEC", "F", "Quality"])
data = swarm.get_between("2016-01-01", "2016-01-03")
ds = data.as_xarray()
ds

### Magnetic models can be evaluated just like with the Swarm MAG datasets

In [ ]:
swarm.set_collection("SW_OPER_AUX_OBSM2_:ABK")
swarm.set_products(
    measurements=["B_NEC"],
    models=["'CHAOS-internal' = 'CHAOS-Core' + 'CHAOS-Static'"],
    residuals=True
)
data = swarm.get_between("2016-01-01", "2016-01-03")
ds = data.as_xarray()

In [ ]:
ds["B_NEC_res_CHAOS-internal"].plot.line(x="Timestamp", col="NEC")

(This roughly shows the disturbance sensed by the observatory due to the magnetospheric and ionospheric sources)